# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

## Imports

In [ ]:
import anthropic
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

## Initialization

In [ ]:
# Load environment variables
load_dotenv()

# Constants
models = {
    "GPT-4o mini": "gpt-4o-mini-2024-07-18",
    "Claude 3.5 Haiku": "claude-3-5-haiku-20241022"
}

# Clients
openai_client = OpenAI()
anthropic_client = anthropic.Anthropic()

## Prompts

In [ ]:
system_message = "You are a very serious assistant that helps customers with their technical questions. \
You are given a question to which you must respond with an easy to understand and concise explanation, \
without requiring previous knowledge on the topic from the customer."

## Functions

In [ ]:
def chat(message, history, model):
    # Remove extra keys ('metadata' and 'options') from history messages as Anthropic does not like them
    cleaned_history = [{"role": message["role"], "content": message["content"]} for message in history]

    response = ""
    if model == "GPT-4o mini":
        messages = [{"role": "system", "content": system_message}] + cleaned_history + [{"role": "user", "content": message}]
        stream = openai_client.chat.completions.create(model=models[model], messages=messages, stream=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            yield response
    elif model == "Claude 3.5 Haiku":
        messages = cleaned_history + [{"role": "user", "content": message}]
        result = anthropic_client.messages.stream(model=models[model], max_tokens=1000, system=system_message, messages=messages)
        with result as stream:
            for chunk in stream.text_stream:
                response += chunk or ""
                yield response

## User Interface (UI)

In [ ]:
with gr.Blocks() as demo:
    model = gr.Dropdown(models, label="Model")

    gr.ChatInterface(
        fn=chat,
        additional_inputs=[model],
        type="messages",
        # save_history=True,
    )
demo.launch()